In [ ]:
!pip install -q tfds-nightly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 35.8 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import tensorflow_datasets as tfds

In [ ]:
from google.colab import files
files.download('/content/drive/MyDrive/ModelCNN_WEBIA.keras')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
  builder = tfds.builder("eurosat")

In [ ]:
ds, info = tfds.load("eurosat",
                    split="train",
                    with_info=True,
                    builder_kwargs={"config": "rgb"})  # On utilise seulement les données RGB (Red, Green, Blue) de Sentinel 2
                                                       # Ce sont les bandes 2,3 et 4

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/eurosat/rgb/incomplete.9AUGGE_2.0.0/eurosat-train.tfrecord*...:   0%|     …

Dataset eurosat downloaded and prepared to /root/tensorflow_datasets/eurosat/rgb/2.0.0. Subsequent calls will reuse this data.


In [ ]:
def preprocess(sample):
    image = sample['image']
    label = sample['label']


    # Ici, nous normalisons les données pour qu'elles soient plus facilement traitable
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

In [ ]:
batch_size = 32
train_size = int(0.8 * info.splits["train"].num_examples)

train_ds = (ds.take(train_size)
            .map(preprocess)
            .batch(batch_size)
            .prefetch(tf.data.AUTOTUNE))

val_ds = (ds.skip(train_size)
          .map(preprocess)
          .batch(batch_size)
          .prefetch(tf.data.AUTOTUNE))

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(64, 64, 3)),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding="same"), # Couche de convolution
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding="same"),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding="same"),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding="same"),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Flatten(), # Transforme les résultat en un seul vecteur
    tf.keras.layers.Dense(256, activation='relu'), # Couche dense (chaque perceprton est relié à un autre de la couche Flatten), l'activation ReLU veut dire qu'on garde que les valeurs positives
    tf.keras.layers.Dropout(0.5), # Met 50% des poids à 0 pour evitter l'overfitting
    tf.keras.layers.Dense(10, activation='softmax') # Couche dense qui donne une proba d'appartenance sur les 10 catégorie (la somme des probas est égale à 1)(c'est à ca que sert l'activation softmax)
])

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(train_ds, validation_data=val_ds, epochs=20)
model.save('/content/drive/MyDrive/ModelCNN_WEBIA.keras')

Epoch 1/20
675/675 ━━━━━━━━━━━━━━━━━━━━ 1467s 2s/step - accuracy: 0.6811 - loss: 0.9060 - val_accuracy: 0.8007 - val_loss: 0.5740
Epoch 2/20
675/675 ━━━━━━━━━━━━━━━━━━━━ 1511s 2s/step - accuracy: 0.7727 - loss: 0.6492 - val_accuracy: 0.8257 - val_loss: 0.5066
Epoch 3/20
675/675 ━━━━━━━━━━━━━━━━━━━━ 1504s 2s/step - accuracy: 0.8221 - loss: 0.5166 - val_accuracy: 0.8572 - val_loss: 0.4120
Epoch 4/20
675/675 ━━━━━━━━━━━━━━━━━━━━ 1521s 2s/step - accuracy: 0.8471 - loss: 0.4435 - val_accuracy: 0.8830 - val_loss: 0.3304
Epoch 5/20
675/675 ━━━━━━━━━━━━━━━━━━━━ 1520s 2s/step - accuracy: 0.8786 - loss: 0.3544 - val_accuracy: 0.8891 - val_loss: 0.3391
Epoch 6/20
675/675 ━━━━━━━━━━━━━━━━━━━━ 1454s 2s/step - accuracy: 0.8951 - loss: 0.2978 - val_accuracy: 0.8733 - val_loss: 0.3964
Epoch 7/20
675/675 ━━━━━━━━━━━━━━━━━━━━ 1512s 2s/step - accuracy: 0.9095 - loss: 0.2585 - val_accuracy: 0.8641 - val_loss: 0.4577
Epoch 8/20
675/675 ━━━━━━━━━━━━━━━━━━━━ 1503s 2s/step - accuracy: 0.9239 - loss: 0.2255 - 